In [1]:
import doctest
import io
import re
from dataclasses import dataclass
from typing import Iterable, List

In [2]:
DATA = "input.txt"

# Part 1

In [3]:
@dataclass
class Card:
    num: int
    winning: set[int]
    numbers: set[int]

    @property
    def matches(self) -> set[int]:
        """Returns the matches for the Card.

        Example:

            >>> c = Card(1, winning={41, 48, 83, 86, 17}, numbers={83, 86, 6, 31, 17, 9, 48, 53})
            >>> sorted(list(c.matches))
            [17, 48, 83, 86]
        """
        return self.winning & self.numbers

    @property
    def points(self):
        """Returns the points for the Card.

        Example:

            >>> c = Card(1, winning={41, 48, 83, 86, 17}, numbers={83, 86, 6, 31, 17, 9, 48, 53})
            >>> c.points
            8

            >>> c = Card(2, winning={13, 32, 20, 16, 61}, numbers={61, 30, 68, 82, 17, 32, 24, 19})
            >>> c.points
            2

            >>> c = Card(3, winning={1, 21, 53, 59, 44}, numbers={69, 82, 63, 72, 16, 21, 14, 1})
            >>> c.points
            2

            >>> c = Card(4, winning={41, 92, 73, 84, 69}, numbers={59, 84, 76, 51, 58, 5, 54, 83})
            >>> c.points
            1

            >>> c = Card(5, winning={87, 83, 26, 28, 32}, numbers={88, 30, 70, 12, 93, 22, 82, 36})
            >>> c.points
            0

            >>> c = Card(6, winning={31, 18, 13, 56, 72}, numbers={74, 77, 10, 23, 35, 67, 36, 11})
            >>> c.points
            0
        """
        total = len(self.matches)
        return 2**max(0, total - 1) if total > 0 else 0

In [4]:
def parse_card(line: str) -> Card:
    """Returns ``Card`` from line.

    Example:

        >>> c = parse_card("Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53")
        >>> c.num == 1
        True
        >>> sorted(list(c.winning))
        [17, 41, 48, 83, 86]
        >>> sorted(list(c.numbers))
        [6, 9, 17, 31, 48, 53, 83, 86]
    """
    m = re.match("Card +(\d+): (.*)", line)
    num = int(m.group(1))
    winning_str, numbers_str = m.group(2).split("|")
    winning = {int(m.group(1)) for m in re.finditer(" *(\d+)", winning_str)}
    numbers = {int(m.group(1)) for m in re.finditer(" *(\d+)", numbers_str)}
    return Card(num, winning, numbers)

In [5]:
def parse_cards(data: io.TextIOBase) -> Iterable[Card]:
    return (parse_card(line) for line in data)

In [6]:
def total_points(cards: Iterable[Card]) -> int:
    """Returns the total points for all cards in ``data``.

    Example:

        >>> data = io.StringIO('''Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
        ... Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
        ... Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
        ... Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
        ... Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
        ... Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
        ... ''')
        >>> total_points(parse_cards(data))
        13
    """
    return sum(c.points for c in cards)

In [7]:
doctest.testmod()

TestResults(failed=0, attempted=20)

In [8]:
with open(DATA, "r") as f:
    print(total_points(parse_cards(f)))

23028


# Part 2

In [9]:
def total_cards(cards: List[Card]) -> int:
    """Returns the total cards for ``data``.

    Example:

        >>> data = io.StringIO('''Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
        ... Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
        ... Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
        ... Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
        ... Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
        ... Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
        ... ''')
        >>> total_cards(list(parse_cards(data)))
        30
    """
    totals = [1]*len(cards)
    for i, card in enumerate(cards):
        for j in range(len(card.matches)):
            totals[i + j + 1] += totals[i]
    return sum(totals)

In [10]:
doctest.testmod()

TestResults(failed=0, attempted=22)

In [11]:
with open(DATA, "r") as f:
    cards = list(parse_cards(f))

In [12]:
print(total_cards(cards))

9236992
